# **Description**
---


1. (60%) Suppose you have 1 M USD and want to invest in US stock market for 10 years (2012-2022). You will decide the portfolio every three months and buy and hold for three months.  At the beginning of every three months: using past 6 months data to compute the risk and correlation of composite stocks of **Nasdaq 100**.
  - Select the first stock which has the smallest average of absolute values of correlations with other stocks
  - Select the second stock which has the smallest absolute values of correlation with the first selected one
  - Select the third stock which has the smallest average of absolute values of correlations with the first two selected stocks.
  - Repeat until you have 10 stocks.
  
  In this assignment, we will take **global minimum variance portfolio (using multiple regression)**

2. (40%)  Compute the maximum drawdown, annual sharpe ratio of your invesment in these 10 years. Try this strategy for the year of 2023 and get the maximum drawdown and annual sharpe ratio. Compare the measures to evaluate whether the performance is consistent.



# SETIABUDHI, Clement Darmawan (SID: 20817485)

In [ ]:
!pip install yfinance

In [ ]:
!pip install yahoo_fin

# Task 1:

In [ ]:
#Importing necessary libraries/tools
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from typing import List
import matplotlib.pyplot as plt
import math
import statsmodels.api as sm
from datetime import datetime, timedelta

In [ ]:
#Downloading necessary data
def download_data(ticker: str, start: str, end: str) -> pd.DataFrame:
    return yf.download(ticker, start=start, end=end)
    if data is None:
        print(f"No data available for {ticker} between {start} and {end}")
    return data

In [ ]:
#Formula to calculate correlation
def calculate_correlation_matrix(df: pd.DataFrame) -> pd.DataFrame:
    return df.pct_change().corr()

In [ ]:
#Formula to calculate variance
def calculate_variance(df: pd.DataFrame) -> pd.Series:
    return df.pct_change().var()

In [ ]:
#Selecting the smallest average of absolute values of correlations
def select_stocks(correlation_matrix: pd.DataFrame, num_stocks: int) -> List[str]:
    selected_stocks = []
    remaining_stocks = list(correlation_matrix.columns)

    for i in range(num_stocks):
        min_corr_stock = None
        min_corr = float('inf')

        for stock in remaining_stocks:
            if i == 0:  # First stock selection
                avg_corr = correlation_matrix[stock].abs().mean() #make it absolute
            elif i == 1:  # Second stock selection
                avg_corr = abs(correlation_matrix.loc[selected_stocks[0], stock])
            else:
                avg_corr = correlation_matrix.loc[selected_stocks, stock].abs().mean()

            if avg_corr < min_corr:
                min_corr = avg_corr
                min_corr_stock = stock

        selected_stocks.append(min_corr_stock)
        remaining_stocks.remove(min_corr_stock)

    return selected_stocks

In [ ]:
#Calculating weights and taking global minimum variance portfolio (using multiple regression)
def calculate_weights(variance: pd.Series, selected_stocks: List[str], market_returns: pd.Series, variances: pd.Series, prices_df: pd.DataFrame) -> pd.Series:
    market_returns = prices_df.mean(axis=1).pct_change().dropna()
    prices_df['market_returns'] = market_returns
    prices_df['market_returns'].fillna(prices_df['market_returns'].mean(), inplace=True)
    prices_df['market_returns'].replace([np.inf, -np.inf], 1e6, inplace=True)

    variances = prices_df.var(axis=0, ddof=1)
    prices_df['variances'] = variances
    prices_df['variances'].fillna(1e6, inplace=True)
    prices_df['variances'].replace([np.inf, -np.inf], 1e6, inplace=True)

    selected_stocks = list(set(selected_stocks).intersection(prices_df.index))
    missing_stocks = set(selected_stocks) - set(prices_df.index)
    if missing_stocks:
        raise ValueError(f"The following stocks are missing from the prices_df DataFrame: {missing_stocks}")

    returns_df = prices_df.loc[selected_stocks].pct_change().dropna()
    selected_variances = variance.loc[selected_stocks]
    selected_variances = selected_variances.fillna(selected_variances.mean())
    selected_variances.replace([np.inf, -np.inf], 1e6, inplace=True)

#Multiple regression
    X = sm.add_constant(prices_df[['market_returns', 'variances']])
    y = pd.Series(0, index=prices_df.index)
    y.loc[selected_stocks] = 1
    model = sm.OLS(y, X, missing='drop')
    results = model.fit()
    weight = results.params[1:]
    weight /= weight.sum()
    return weight

In [ ]:
#Preparation
risk_free_rate = 0.02
start_date = '2012-01-01'
end_date = '2022-12-31'
initial = 1000000 #total amount
variance = pd.Series
selected_stocks = List[str]
market_returns = pd.Series
variances = pd.Series

#Importing Nasdaq 100 tickers
nasdaq100list = pd.read_html("https://en.wikipedia.org/wiki/Nasdaq-100")
tickers = nasdaq100list[4]["Ticker"].tolist()

num_stocks = 10
quarters = pd.date_range(start=start_date, end=end_date, freq='Q', closed='right')
portfolio_value = initial
max_drawdown = 0
all_returns= []
wealth_process = pd.DataFrame({'date': [start_date], 'wealth': [initial]})
wealth_process = wealth_process.set_index('date')
all_wealth_processes = [] # Define an empty list to store wealth processes


# Download Nasdaq 100 data
ticker = "^NDX"
nasdaq100_data = download_data(ticker, start_date, end_date)
nasdaq100_prices = nasdaq100_data['Adj Close']
nasdaq100_returns = nasdaq100_prices.pct_change().dropna()

[*********************100%***********************]  1 of 1 completed


<ipython-input-104-57d5c479d05d>:16: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  quarters = pd.date_range(start=start_date, end=end_date, freq='Q', closed='right')


In [ ]:
#Processing the data using all of the functions
wealth_process = pd.DataFrame({'date': [start_date], 'wealth': [initial]})
wealth_process = wealth_process.set_index('date')

for quarter_end in quarters:
    start = (quarter_end - pd.DateOffset(months=6)).strftime('%Y-%m-%d')
    end = quarter_end.strftime('%Y-%m-%d')

    data = {}
    for ticker in tickers:
      try:
        data[ticker] = download_data(ticker, start, end)
      except BaseException:
        print(f"Error downloading data for {ticker}")
        data[ticker] = None

    prices_df = pd.DataFrame({ticker: df['Adj Close'] for ticker, df in data.items() if df is not None})

    correlation_matrix = calculate_correlation_matrix(prices_df)
    variance = calculate_variance(prices_df)

    selected_stocks = select_stocks(correlation_matrix, num_stocks)

    market_returns = prices_df.mean(axis=1).pct_change().dropna()
    prices_df['market_returns'] = market_returns
    prices_df['market_returns'].fillna(prices_df['market_returns'].mean(), inplace=True)
    prices_df['market_returns'].replace([np.inf, -np.inf], 1e6, inplace=True)

    variances = prices_df.var(axis=0, ddof=1)
    prices_df['variances'] = variances
    prices_df['variances'].fillna(1e6, inplace=True)
    selected_variances = variance.loc[selected_stocks]
    selected_variances = selected_variances.fillna(selected_variances.mean())
    selected_variances.replace([np.inf, -np.inf], 1e6, inplace=True)

    weights = calculate_weights(variances, selected_stocks, prices_df['market_returns'], prices_df['variances'], prices_df)

    start_next_quarter = (quarter_end + pd.DateOffset(days=1)).strftime('%Y-%m-%d')
    end_next_quarter = (quarter_end + pd.DateOffset(months=3)).strftime('%Y-%m-%d')

    next_quarter_data = {ticker: download_data(ticker, start_next_quarter, end_next_quarter) for ticker in selected_stocks}
    next_quarter_prices_df = pd.DataFrame({ticker: df['Adj Close'] for ticker, df in next_quarter_data.items()})
    returns = next_quarter_prices_df.pct_change().dropna()

    # Compute daily returns
    daily_returns = (weights * returns).sum(axis=1)
    all_returns.extend(daily_returns)

    # Update wealth process
    end_date = pd.to_datetime(end_next_quarter).date()
    start_date = end_date - pd.DateOffset(months=3) + pd.DateOffset(days=1)

    wealth_period = nasdaq100_returns.loc[start_date:end_date] + 1
    wealth_process_quarter = wealth_period.cumprod() * portfolio_value
    wealth_process_quarter = pd.DataFrame({'date': wealth_process_quarter.index, 'wealth': wealth_process_quarter.values})
    wealth_process_quarter = wealth_process_quarter.set_index('date')
    wealth_process = wealth_process.append(wealth_process_quarter)

    portfolio_value = wealth_process.iloc[-1]['wealth']

# Task 2:

In [ ]:
#Function to calculate maximum drawdown
def calculate_max_drawdown(wealth_process: pd.DataFrame) -> float:
    max_drawdown = 0
    peak = wealth_process.iloc[0]['wealth']
    for _, row in wealth_process.iterrows():
        if row['wealth'] > peak:
            peak = row['wealth']
        drawdown = (peak - row['wealth']) / peak
        if drawdown > max_drawdown:
            max_drawdown = drawdown
    return max_drawdown
    if len(wealth_process) == 0:
        return 0

In [ ]:
#Function to calculate annual sharpe ratio
def calculate_sharpe_ratio(returns: np.ndarray, risk_free_rate: float) -> float:
    excess_returns = returns - risk_free_rate
    average_excess_return = np.mean(excess_returns)
    std_deviation = np.std(returns)
    sharpe_ratio = average_excess_return / std_deviation
    return sharpe_ratio

In [ ]:
def get_nasdaq100_tickers():
    nasdaq100 = yf.Ticker("^NDX")
    nasdaq100_constituents = nasdaq100.sustainability['constituents'].split(' ')
    return nasdaq100_constituents[:-1] # Remove the last empty element

In [ ]:
#Calculating maximum drawdown, annual sharpe ratio of invesment during 2012-2022
portfolio_returns = wealth_process['wealth'].pct_change().fillna(0)
max_drawdown = calculate_max_drawdown(wealth_process)
sharpe_ratio = calculate_sharpe_ratio(returns, risk_free_rate)

print(f"Portfolio Value after 10 years: ${wealth_process.iloc[-1]['wealth']:.2f}")
print(f"Maximum Drawdown for 2012-2022: {max_drawdown * 100:.2f}%")
print(f"Annual Sharpe Ratio for 2012-2022: ", sharpe_ratio)

Portfolio Value after 10 years: $3865793.76
Maximum Drawdown for 2012-2022: 36.28%
Annual Sharpe Ratio for 2012-2022:  KHC    -1.615193
WBD    -0.358638
ENPH   -0.678933
REGN   -0.940855
PDD    -0.521618
SGEN   -0.374335
FANG   -0.819191
ATVI   -1.099388
DLTR   -1.214224
CEG    -1.105293
dtype: float64


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [ ]:
#Calculating maximum drawdown, annual sharpe ratio of invesment during 2023

#Preparation
risk_free_rate = 0.02
start_date = '2023-01-01'
end_date = datetime.today().strftime('%Y-%m-%d')
initial = 1000000 #total amount
variance = pd.Series
selected_stocks = List[str]
market_returns = pd.Series
variances = pd.Series

#Importing Nasdaq 100 tickers
nasdaq100list = pd.read_html("https://en.wikipedia.org/wiki/Nasdaq-100")
tickers = nasdaq100list[4]["Ticker"].tolist()

num_stocks = 10
quarters = pd.date_range(start=start_date, end=end_date, freq='Q', closed='right')
portfolio_value = initial
max_drawdown = 0
all_returns= []
wealth_process = pd.DataFrame({'date': [start_date], 'wealth': [initial]})
wealth_process = wealth_process.set_index('date')
all_wealth_processes = [] # Define an empty list to store wealth processes


# Download Nasdaq 100 data
ticker = "^NDX"
nasdaq100_data = download_data(ticker, start_date, end_date)
nasdaq100_prices = nasdaq100_data['Adj Close']
nasdaq100_returns = nasdaq100_prices.pct_change().dropna()

#Processing the data using all of the functions
wealth_process = pd.DataFrame({'date': [start_date], 'wealth': [initial]})
wealth_process = wealth_process.set_index('date')

for quarter_end in quarters:
    start = (quarter_end - pd.DateOffset(months=6)).strftime('%Y-%m-%d')
    end = quarter_end.strftime('%Y-%m-%d')

    data = {}
    for ticker in tickers:
      try:
        data[ticker] = download_data(ticker, start, end)
      except BaseException:
        print(f"Error downloading data for {ticker}")
        data[ticker] = None

    prices_df = pd.DataFrame({ticker: df['Adj Close'] for ticker, df in data.items() if df is not None})

    correlation_matrix = calculate_correlation_matrix(prices_df)
    variance = calculate_variance(prices_df)

    selected_stocks = select_stocks(correlation_matrix, num_stocks)

    market_returns = prices_df.mean(axis=1).pct_change().dropna()
    prices_df['market_returns'] = market_returns
    prices_df['market_returns'].fillna(prices_df['market_returns'].mean(), inplace=True)
    prices_df['market_returns'].replace([np.inf, -np.inf], 1e6, inplace=True)

    variances = prices_df.var(axis=0, ddof=1)
    prices_df['variances'] = variances
    prices_df['variances'].fillna(1e6, inplace=True)
    selected_variances = variance.loc[selected_stocks]
    selected_variances = selected_variances.fillna(selected_variances.mean())
    selected_variances.replace([np.inf, -np.inf], 1e6, inplace=True)

    weights = calculate_weights(variances, selected_stocks, prices_df['market_returns'], prices_df['variances'], prices_df)

    start_next_quarter = (quarter_end + pd.DateOffset(days=1)).strftime('%Y-%m-%d')
    end_next_quarter = (quarter_end + pd.DateOffset(months=3)).strftime('%Y-%m-%d')

    next_quarter_data = {ticker: download_data(ticker, start_next_quarter, end_next_quarter) for ticker in selected_stocks}
    next_quarter_prices_df = pd.DataFrame({ticker: df['Adj Close'] for ticker, df in next_quarter_data.items()})
    returns = next_quarter_prices_df.pct_change().dropna()

    # Compute daily returns
    daily_returns = (weights * returns).sum(axis=1)
    all_returns.extend(daily_returns)

    # Update wealth process
    end_date = pd.to_datetime(end_next_quarter).date()
    start_date = end_date - pd.DateOffset(months=3) + pd.DateOffset(days=1)

    wealth_period = nasdaq100_returns.loc[start_date:end_date] + 1
    wealth_process_quarter = wealth_period.cumprod() * portfolio_value
    wealth_process_quarter = pd.DataFrame({'date': wealth_process_quarter.index, 'wealth': wealth_process_quarter.values})
    wealth_process_quarter = wealth_process_quarter.set_index('date')
    wealth_process = wealth_process.append(wealth_process_quarter)

    portfolio_value = wealth_process.iloc[-1]['wealth']

portfolio_returns = wealth_process['wealth'].pct_change().fillna(0)
max_drawdown = calculate_max_drawdown(wealth_process)
sharpe_ratio = calculate_sharpe_ratio(returns, risk_free_rate)

print(f"Portfolio Value for 2023: ${wealth_process.iloc[-1]['wealth']:.2f}")
print(f"Maximum Drawdown for 2023: {max_drawdown * 100:.2f}%")
print(f"Annual Sharpe Ratio for 2023: ", sharpe_ratio)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

<ipython-input-114-e8a5a4ad270c>:18: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  quarters = pd.date_range(start=start_date, end=end_date, freq='Q', closed='right')



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%*******

<ipython-input-114-e8a5a4ad270c>:89: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  wealth_process = wealth_process.append(wealth_process_quarter)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
